In [3]:
from ipywidgets import Dropdown, interact, IntSlider, FloatSlider
from IPython.display import display, clear_output
import plotly.graph_objects as go
import pandas as pd
# === CLASSES ===
class TimeSeries:
    def __init__(self, hourly_values):
        self.hourly_values = hourly_values
    def __getitem__(self, hour):
        return self.hourly_values[hour % 24]

class Battery:
    def __init__(self, battery_capacity_kWh, initial_soc_ratio):
        self.capacity_kWh = battery_capacity_kWh
        self.initial_soc_ratio = initial_soc_ratio
        self.current_energy_kWh = initial_soc_ratio * battery_capacity_kWh

class Vehicle:
    def __init__(self, battery, arrival_hour, departure_hour, target_soc_ratio):
        self.battery = battery
        self.arrival_hour = arrival_hour
        self.departure_hour = departure_hour
        self.target_soc_ratio = target_soc_ratio

# === DONNÉES PV & PROFILS ===
pv_data_by_month = {
    "January":  [0,0,0,0,0,0,0.156,0.290,0.336,0.368,0.400,0.421,0.380,0.347,0.144,0,0,0,0,0,0,0,0,0],
    "February": [0,0,0,0,0,0.001,0.260,0.349,0.432,0.451,0.490,0.486,0.483,0.424,0.317,0.060,0,0,0,0,0,0,0,0],
    "March":    [0,0,0,0,0.003,0.223,0.389,0.497,0.572,0.604,0.627,0.599,0.565,0.523,0.439,0.309,0.001,0,0,0,0,0,0,0],
    "April":    [0,0,0,0.0002,0.171,0.361,0.516,0.617,0.664,0.678,0.684,0.648,0.617,0.538,0.495,0.383,0.164,0,0,0,0,0,0,0],
    "May":      [0,0,0,0.068,0.248,0.402,0.542,0.639,0.691,0.692,0.682,0.646,0.604,0.540,0.508,0.419,0.266,0.003,0,0,0,0,0,0],
    "June":     [0,0,0,0.118,0.321,0.498,0.632,0.737,0.773,0.765,0.752,0.733,0.676,0.609,0.572,0.493,0.351,0.095,0,0,0,0,0,0],
    "July":     [0,0,0,0.085,0.311,0.506,0.646,0.747,0.804,0.808,0.797,0.778,0.722,0.674,0.621,0.532,0.375,0.079,0,0,0,0,0,0],
    "August":   [0,0,0,0.003,0.203,0.399,0.556,0.664,0.744,0.769,0.756,0.737,0.723,0.647,0.579,0.478,0.280,0.008,0,0,0,0,0,0],
    "September":[0,0,0,0.076,0.263,0.400,0.546,0.640,0.678,0.678,0.645,0.606,0.550,0.477,0.325,0.019,0,0,0,0,0,0,0,0],
    "October":  [0,0,0,0,0,0.147,0.237,0.332,0.414,0.517,0.551,0.524,0.499,0.429,0.305,0.021,0,0,0,0,0,0,0,0],
    "November": [0,0,0,0,0,0.010,0.163,0.227,0.254,0.307,0.327,0.342,0.329,0.258,0.071,0,0,0,0,0,0,0,0,0],
    "December": [0,0,0,0,0,0,0.123,0.235,0.258,0.275,0.301,0.302,0.300,0.245,0.0003,0,0,0,0,0,0,0,0,0]
}
# Conversion irradiation → production réelle (kWh par panneau de 2 m², rendement 75 %)
pv_output_by_month = {
    month: [round(value * 2.0 * 0.75, 3) for value in hourly]
    for month, hourly in pv_data_by_month.items()
}

user_profiles = {
    "Evening Users": [0.3, 0.2, 0.2, 0.2, 0.1, 0.2, 0.3, 0.5, 0.6, 0.7, 0.8, 0.9, 1.2, 1.5, 1.8, 2.0, 2.1, 2.2, 2.5, 2.0, 1.2, 0.8, 0.5, 0.4],
    "Late Afternoon Peakers": [0.2]*8 + [0.5, 0.8, 1.5, 2.0, 2.2, 2.0, 1.8, 1.5, 1.0, 0.6, 0.4, 0.2, 0.2, 0.2, 0.2, 0.2],
    "Coffee Makers": [0.1, 0.1, 0.3, 1.0, 1.5, 1.2, 1.0, 0.8, 0.6, 0.5, 0.4, 0.3, 0.4, 0.6, 0.8, 1.0, 1.1, 1.2, 1.1, 0.8, 0.6, 0.4, 0.3, 0.2],
    "Night Owls": [0.2]*18 + [0.8, 1.2, 1.5, 1.6, 1.3, 1.0],
    "Morning Glory": [1.2, 1.5, 1.6, 1.4, 1.2, 1.0, 0.8, 0.5, 0.3, 0.2, 0.2, 0.2, 0.3, 0.4, 0.6, 0.7, 0.6, 0.5, 0.4, 0.4, 0.3, 0.3, 0.3, 0.3]
}

# === SIMULATION ===
def run_simulation(month, profile_name, arrival_hour, departure_hour, initial_soc, target_soc):
    clear_output(wait=True)

    BATTERY_CAPACITY_KWH = 70
    MAX_KWH_PER_HOUR = 11
    MIN_SOC_RATIO = 0.2
    TARIFF_PER_KWH = 0.25

    house_demand = TimeSeries(user_profiles[profile_name])
    pv_generation = TimeSeries(pv_output_by_month[month])
    battery = Battery(BATTERY_CAPACITY_KWH, initial_soc)
    vehicle = Vehicle(battery, arrival_hour, departure_hour, target_soc)

    if arrival_hour < departure_hour:
        hours_range = range(arrival_hour, departure_hour)
    else:
        hours_range = list(range(arrival_hour, 24)) + list(range(0, departure_hour))

    soc_each_hour_kWh = [None] * 24
    battery_flow_kWh = [0] * 24
    net_load_kWh = [0] * 24
    current_soc_kWh = battery.current_energy_kWh
    total_cost_no_v2h = 0
    total_cost_with_v2h = 0

    for h in hours_range:
        h_mod = h % 24
        pv = pv_generation[h_mod]
        demand = house_demand[h_mod]
        battery_effect = 0

        cost_no_v2h = max(demand - pv, 0) * TARIFF_PER_KWH
        total_cost_no_v2h += cost_no_v2h

        if h == arrival_hour:
            soc_each_hour_kWh[h_mod] = current_soc_kWh
            continue

        target_energy = vehicle.target_soc_ratio * BATTERY_CAPACITY_KWH
        energy_needed = target_energy - current_soc_kWh
        max_discharge = current_soc_kWh - MIN_SOC_RATIO * BATTERY_CAPACITY_KWH
        house_need = demand - pv
        hours_left = len([hr for hr in hours_range if hr >= h])
        max_recharge_possible = (hours_left - 1) * MAX_KWH_PER_HOUR

        # === Recharge dès qu’on en a besoin et dès que possible
        if energy_needed > 0:
            if pv > demand:
                surplus_pv = pv - demand
                battery_effect = min(energy_needed, surplus_pv, MAX_KWH_PER_HOUR)
            else:
                battery_effect = min(energy_needed, MAX_KWH_PER_HOUR)

        # === Décharge améliorée : fluide et cohérente
        elif house_need > 0 and max_discharge > 0:
            remaining_energy = current_soc_kWh - min(house_need, max_discharge, MAX_KWH_PER_HOUR)
            if remaining_energy + max_recharge_possible >= target_energy:
                battery_effect = -min(house_need, max_discharge, MAX_KWH_PER_HOUR)

        # Mise à jour état batterie
        current_soc_kWh += battery_effect
        soc_each_hour_kWh[h_mod] = current_soc_kWh
        battery_flow_kWh[h_mod] = round(battery_effect, 2)
        net = demand - pv + battery_effect
        net_load_kWh[h_mod] = round(max(net, 0), 2)
        total_cost_with_v2h += net_load_kWh[h_mod] * TARIFF_PER_KWH

    soc_each_hour_kWh[vehicle.departure_hour] = current_soc_kWh
    soc_percent = [round(100 * s / BATTERY_CAPACITY_KWH, 2) if s is not None else None for s in soc_each_hour_kWh]
    hours = [f"{h:02d}:00" for h in range(24)]

    savings = round(total_cost_no_v2h - total_cost_with_v2h, 2)
    final_soc_pct = round(current_soc_kWh / BATTERY_CAPACITY_KWH * 100, 2)
    target_soc_pct = round(vehicle.target_soc_ratio * 100, 1)

    if current_soc_kWh < target_energy:
        print(f"⚠️ SoC target NOT reached ! ({final_soc_pct}% < {target_soc_pct}%)")
    else:
        print(f"✅ SoC cible atteint ({final_soc_pct}%)")

    fig = go.Figure()
    fig.add_trace(go.Scatter(x=hours, y=pv_generation.hourly_values, name="PV (kW)", mode="lines+markers", line=dict(color="orange")))
    fig.add_trace(go.Scatter(x=hours, y=house_demand.hourly_values, name="House Demand (kW)", mode="lines+markers", line=dict(color="gray")))
    fig.add_trace(go.Scatter(x=hours, y=net_load_kWh, name="Net Load (kW)", mode="lines+markers", line=dict(color="black")))
    fig.add_trace(go.Scatter(x=hours, y=battery_flow_kWh, name="Battery Flow (kWh)", mode="lines+markers", line=dict(color="brown", dash="dot")))
    fig.add_trace(go.Scatter(x=hours, y=soc_percent, name="SoC (%)", mode="lines+markers", line=dict(color="blue", dash="dot"), yaxis="y2"))
    fig.add_trace(go.Scatter(x=[hours[arrival_hour]], y=[initial_soc * 100], mode='markers', marker=dict(color='green', size=12), name="Arrival", yaxis="y2"))
    fig.add_trace(go.Scatter(x=[hours[departure_hour]], y=[final_soc_pct], mode='markers', marker=dict(color='red', size=12), name="Departure", yaxis="y2"))

    fig.update_layout(
        title=f"<b>⚡ Smart Recharge V2H – {month} | {profile_name} ｜💶 Savings: {savings}€ ｜🔋 Final SoC: {final_soc_pct}%</b>",
        xaxis=dict(title="Hour"),
        yaxis=dict(title="Power (kW)", side="left"),
        yaxis2=dict(title="State of Charge (%)", overlaying="y", side="right", range=[0, 100]),
        height=500,
        template="plotly_white",
        legend=dict(orientation="h", yanchor="top", y=1.1, xanchor="center", x=0.5),
        margin=dict(t=80, b=60, l=60, r=60)
    )

    display(fig)






    # === CALCUL FLEXIBILITÉ & SELF-SUFFICIENCY ===
    battery_capacity_kWh = BATTERY_CAPACITY_KWH

    summary_df = pd.DataFrame({
        "hour": list(range(24)),
        "house_demand": house_demand.hourly_values,
        "pv_generation": pv_generation.hourly_values,
        "battery_flow": battery_flow_kWh
    })

    flex_kWh = abs(summary_df[summary_df["battery_flow"] < 0]["battery_flow"].sum())
    flex_pct = round(100 * flex_kWh / battery_capacity_kWh, 2)

    covered_by_pv = summary_df[["pv_generation", "house_demand"]].min(axis=1)
    net_demand_after_pv = summary_df["house_demand"] - covered_by_pv
    battery_help = summary_df["battery_flow"].clip(upper=0).abs()
    covered_total = covered_by_pv + battery_help
    self_suff_pct = round(100 * covered_total.sum() / summary_df["house_demand"].sum(), 2)
    # Nouveaux indicateurs
    energy_charged_kWh = summary_df[summary_df["battery_flow"] > 0]["battery_flow"].sum()
    energy_discharged_kWh = summary_df[summary_df["battery_flow"] < 0]["battery_flow"].abs().sum()

    # Affichage final
    print(f"\n🔁 Flexibility used : {round(flex_kWh, 2)} kWh ({flex_pct} % of the battery)")
    print(f"🏠 Autonomy energetic (self-sufficiency) : {self_suff_pct} %")
    print(f"🔋 Energy charged : {round(energy_charged_kWh, 2)} kWh")
    print(f"🔻 Energy discharged : {round(energy_discharged_kWh, 2)} kWh")


   

# === INTERFACE ===
from ipywidgets import Dropdown, IntSlider, FloatSlider, interact
from IPython.display import display, clear_output
import plotly.graph_objects as go

interact(
    run_simulation,
    month=Dropdown(options=list(pv_data_by_month.keys()), value="July", description="🗓 Month :"),
    profile_name=Dropdown(options=list(user_profiles.keys()), value="Evening Users", description="👤 Profile :"),
    arrival_hour=IntSlider(min=0, max=23, step=1, value=8, description="🔌 Arrival"),
    departure_hour=IntSlider(min=0, max=23, step=1, value=19, description="🔋 Departure"),
    initial_soc=FloatSlider(min=0.2, max=0.8, step=0.05, value=0.4, description="SoC init."),
    target_soc=FloatSlider(min=0.3, max=1.0, step=0.05, value=0.8, description="SoC target")
)



interactive(children=(Dropdown(description='🗓 Month :', index=6, options=('January', 'February', 'March', 'Apr…

<function __main__.run_simulation(month, profile_name, arrival_hour, departure_hour, initial_soc, target_soc)>